In [1]:
# Check to see if GPU is being used.

import torch
print(torch.__version__)
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA (GPU support) is available in this environment!")
else:
    print("CUDA (GPU support) is NOT available.")

2.1.0a0+29c30b1
True
CUDA (GPU support) is available in this environment!


In [2]:
tensor = torch.tensor([1, 2, 3])
print(tensor.device)

cpu


https://huggingface.co/docs/transformers/quicktour

In [1]:
%%time
from transformers import pipeline

Start by creating an instance of `pipeline()` and specify a task you want to use it for. e.g. use `pipeline()` for sentiment analysis as an example:

In [2]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

The `pipeline()` downloads and caches a default pretrained model and tokenizer for sentiment analysis. Now you can use classifier on your target text:

In [3]:
classifier("We are very happy to show you the 🤗 Transformers library.")

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

In [4]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


In [5]:
speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

Load an audio dataset to iterate over. For example, load [MlnDS-14](https://huggingface.co/datasets/PolyAI/minds14) dataset.

In [6]:
from datasets import load_dataset, Audio

In [7]:
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")

Generating train split:   0%|          | 0/563 [00:00<?, ? examples/s]

You need to make sure sampling rate of dataset matches sampling rate [`facebook/wav2vec-base-960h`](https://huggingface.co/facebook/wav2vec2-base-960h) was trained on:

In [9]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))

The audio files are automatically loaded and resampled when calling the "audio" column. Extract raw waveform arrays from first 4 samples and pass it as a list to the pipeline:

In [10]:
result = speech_recognizer(dataset[:4]["audio"])
print([d["text"] for d in result])

['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', "FODING HOW I'D SET UP A JOIN TO HET WITH MY WIFE AND WHERE THE AP MIGHT BE", "I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE AP SO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP AND I'M MISSING SOMETHING UQUETTE HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS", 'HOW DO I THURN A JOIN A COUNT']


For larger datasets where inputs are big, you'll want to pass a generator instead of a list to load inputs into memory. Look at [pipeline API reference](https://huggingface.co/docs/transformers/main_classes/pipelines) for more information.

### Use another model and tokenizer in the pipeline

In [4]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

# Pytorch, use AutoModelForSequenceClassification to load pretrained model, and associated tokenizer.
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:835: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
if torch.cuda.is_available():
    model = model.cuda()
    print("Moved model to the GPU")
else:
    print("Model still on CPU")

Moved model to the GPU


Specify the model and tokenizer in the `pipeline()` and now you can apply the classifier on French text:

In [12]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [13]:
classifier("Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.")

[{'label': '5 stars', 'score': 0.7272652387619019}]

If you can't find a model for your use-case, you'll need to finetune a pretrained model on your data.

* [finetuning tutorial](https://huggingface.co/docs/transformers/training)

# AutoClass

[AutoClass](https://huggingface.co/docs/transformers/model_doc/auto) is a shortcut that automatically retrieves the architecture of a pretrained model from its name or path.

The tokenizer returns a dictionary containing:
* [input_ids](https://huggingface.co/docs/transformers/glossary#input-ids) numerical representations of your tokens
* [attention_mask](https://huggingface.co/docs/transformers/.glossary#attention-mask) indicate which tokens should be attended to

In [14]:
encoding = tokenizer("We are very happy to show you the 🤗 Transformers library.")
print(encoding)

{'input_ids': [101, 11312, 10320, 12495, 19308, 10114, 11391, 10855, 10103, 100, 58263, 13299, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [15]:
# Pytorch
pt_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",)
print(pt_batch)

{'input_ids': tensor([[  101, 11312, 10320, 12495, 19308, 10114, 11391, 10855, 10103,   100,
         58263, 13299,   119,   102],
        [  101, 11312, 18763, 10855, 11530,   112,   162, 39487, 10197,   119,
           102,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}


Check out how to use an
* [AutoImageProcessor](https://huggingface.co/docs/transformers/v4.36.1/en/model_doc/auto#transformers.AutoImageProcessor)
* [AutoFeatureExtractor](https://huggingface.co/docs/transformers/v4.36.1/en/model_doc/auto#transformers.AutoFeatureExtractor)
* [AutoProcessor](https://huggingface.co/docs/transformers/v4.36.1/en/model_doc/auto#transformers.AutoProcessor)

to preprocess image, audio, and multimodal inputs.

### AutoModel

For text (or sequence) classification, you should load [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/v4.36.1/en/model_doc/auto#transformers.AutoModelForSequenceClassification)

Pass preprocessed batch of inputs directly to the model. Unpack dictionary by adding **:

In [16]:
pt_outputs = model(**pt_batch)

The model outputs the final activations in the logits attribute. Apply softmax function to retrieve probabilities.

In [17]:
from torch import nn

In [18]:
pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
print(pt_predictions)

tensor([[0.0021, 0.0018, 0.0115, 0.2121, 0.7725],
        [0.2084, 0.1826, 0.1969, 0.1755, 0.2365]], grad_fn=<SoftmaxBackward0>)


#### Save a model

Once your model is fine-tuned, you can save it with its tokenizer using [PreTrainedModel.save_pretrained()](https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/model#transformers.PreTrainedModel.save_pretrained)

In [20]:
pt_save_directory = "./pt_save_pretrained"
tokenizer.save_pretrained(pt_save_directory)
model.save_pretrained(pt_save_directory)

In [21]:
# When ready to use the model again, reload it
pt_model = AutoModelForSequenceClassification.from_pretrained("./pt_save_pretrained")

#### Custom model builds

You can modify the model's configuration class to change how a model is built. The configuration specifies a model's attributes, such as number of hidden layers or attention heads.You start from scratch when you initialize a model from a custom configuration class. The model attributes are randomly initialized, and you'll need to train the model before you can use it to get meaningful results.

Within [`AutoConfig.from_pretrained()`](https://huggingface.co/docs/transformers/v4.36.1/en/model_doc/auto#transformers.AutoConfig.from_pretrained) you can specify attribute you want to change, such as number of attention heads:

In [5]:
from transformers import AutoConfig, AutoModel, AutoModelForSequenceClassification

In [2]:
my_config = AutoConfig.from_pretrained("distilbert-base-uncased", n_heads=12)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Create a model from your custom configuration:

In [4]:
my_model = AutoModel.from_config(my_config)

#### Trainer - a PyTorch optimized training loop

In [6]:
# Start with a PreTrainedModel or torch.nn.Module
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import TrainingArguments, AutoTokenizer, DataCollatorWithPadding

In [7]:
# TrainingArguments contains the model hyperparameters
training_args = TrainingArguments(
    output_dir="./example_save_folder/",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2)

# Load preprocessing class like a tokenizer, image processor, feature extractor, or processor
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [8]:
# Load a dataset

from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")

In [10]:
# Create a function to tokenize dataset.
def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])

# Then apply it over entire dataset with map:

dataset = dataset.map(tokenize_dataset, batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [11]:
# A DataCollatorWithPadding to create a batch of examples from your dataset:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
from transformers import Trainer

Now gather all these classes in [Trainer](https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.Trainer)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator) # doctest: + SKIP

When you're ready, call [`train()`](https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.Trainer.train) to start training:

In [14]:
%%time
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: ERROR API key must be 40 characters long, yours was 14
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.706700
1000,0.670300
1500,0.580400
2000,0.544500


CPU times: user 3min 34s, sys: 18.1 s, total: 3min 52s
Wall time: 5min 51s


TrainOutput(global_step=2134, training_loss=0.6165462447419385, metrics={'train_runtime': 350.8161, 'train_samples_per_second': 48.629, 'train_steps_per_second': 6.083, 'total_flos': 389237040539508.0, 'train_loss': 0.6165462447419385, 'epoch': 2.0})